In [1]:
print("ok")

ok


In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import math
from typing import Tuple, Optional
import librosa
import transformers

/home/bishwa/Unversity/Audio_LLM_Memory/audio_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch.nn as nn
import torch
from MemoryModule.conponents.LinearAttention import LinearAttentionMem
from MemoryModule.conponents.Mom import MOM
from MemoryModule.utils.common import  bias_term_adjust
B = 8
S = 186
n_text_state = 1280
# Dummy hidden states
x = torch.arange(B*S*n_text_state).float().reshape(B,S,n_text_state)
print("x =", x.shape)
n_heads = 8
num_memories = 8
mom = MOM(n_text_state=n_text_state,
          n_heads=n_heads,
          num_memories=num_memories,
          bias_term_adjust=bias_term_adjust,
          LinearAttentionMem=LinearAttentionMem)
mom(x)


x = torch.Size([8, 186, 1280])
final_output= torch.Size([8, 186, 1280])


tensor([[[ 8.0532e+12,  5.6196e+13, -5.5353e+12,  ...,  1.0981e+13,
          -9.2761e+12,  3.0630e+13],
         [ 4.7144e+13,  1.4751e+14, -1.6219e+13,  ...,  4.4794e+13,
           4.5639e+12,  8.8861e+13],
         [ 8.6234e+13,  2.3882e+14, -2.6903e+13,  ...,  7.8606e+13,
           1.8404e+13,  1.4709e+14],
         ...,
         [ 7.1616e+15,  1.6766e+16, -1.9607e+15,  ...,  6.1987e+15,
           2.5235e+15,  1.0687e+16],
         [ 7.2007e+15,  1.6858e+16, -1.9714e+15,  ...,  6.2325e+15,
           2.5373e+15,  1.0745e+16],
         [ 7.2398e+15,  1.6949e+16, -1.9821e+15,  ...,  6.2663e+15,
           2.5511e+15,  1.0803e+16]],

        [[ 5.1084e+16,  1.1942e+17, -1.3895e+16,  ...,  4.4076e+16,
           1.8059e+16,  7.6088e+16],
         [ 5.1358e+16,  1.2006e+17, -1.3970e+16,  ...,  4.4313e+16,
           1.8157e+16,  7.6496e+16],
         [ 5.1632e+16,  1.2070e+17, -1.4044e+16,  ...,  4.4549e+16,
           1.8254e+16,  7.6904e+16],
         ...,
         [ 1.0129e+17,  2

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
print(torch.cuda.device_count())

1


In [1]:
!nvidia-smi

Tue Dec 23 18:29:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        Off |   00000000:02:00.0 Off |                  N/A |
|  0%   28C    P8             10W /  575W |     192MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [36]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))

2.9.1+cu130
13.0
NVIDIA GeForce RTX 5090


In [3]:
def bias_term_adjust(router_weight, softmax, avg_load_per_expert, num_memories, bias, u=0.001):
    """
    router_weight: [B, S, E]   (E = num_memories)
    bias:          [1, E] or [B, E]
    """

    # Get topk values & indices ----
    router_load_weight, top_memories_indx = torch.topk(
        router_weight, 2, dim=-1
    )


    # Scatter into full expert dimension ----
    routing_weights_full = torch.full(
        (router_weight.shape[0], router_weight.shape[1], num_memories),
        float('-inf'),
        device=router_weight.device,
        dtype=router_weight.dtype
    ).scatter(-1, top_memories_indx, router_load_weight)

    # Normalize
    routing_weights_full_norm = softmax(routing_weights_full)

    # Count routed tokens per expert
    token_routed_each_expert = torch.count_nonzero(routing_weights_full_norm, dim=-2)

    # Compute load violation
    load_violation = avg_load_per_expert - token_routed_each_expert
    # Shape: [B, E]

    # Aggregate over batch
    # Instead of updating using last batch element: load_violation[-1]
    # we do MEAN over batch = batch gradient estimate
    load_signal = load_violation.mean(dim=0)     # shape [E]

    # compute sign
    direction = torch.sign(load_signal)

    # Apply router weight update (vectorized)
    # router_weight[:,:,idx] += load * bias[:,idx]
    router_weight = router_weight + direction.unsqueeze(0).unsqueeze(0) * bias

    # Bias update like gradient descent
    # bias = bias + u * load_signal
    bias = bias + u * direction.unsqueeze(0)

    # Recompute new routing weights
    router_load_weight, top_memories_indx = torch.topk(
        router_weight, 2, dim=-1
    )

    routing_weights_full = torch.full(
        (router_weight.shape[0], router_weight.shape[1], num_memories),
        float('-inf'),
        device=router_weight.device,
        dtype=router_weight.dtype
    ).scatter(-1, top_memories_indx, router_load_weight)

    routing_weights_full_norm = softmax(routing_weights_full)

    return routing_weights_full_norm, bias

In [2]:
class MHARouting(nn.Module):
    def __init__(
        self,
        n_text_state: int,
        n_heads: int = 8,
        prenorm: bool = True,
    ):
        super().__init__()

        self.n_text_state = n_text_state
        self.n_heads = n_heads

        # Multihead Attention
        self.mha = nn.MultiheadAttention(
            embed_dim=n_text_state,
            num_heads=n_heads,
            batch_first=True
        )

        # LayerNorm
        self.input_norm = nn.LayerNorm(n_text_state) if prenorm else nn.Identity()
        self.output_norm = nn.Identity() if prenorm else nn.LayerNorm(n_text_state)

        # Router
        self.router_norm = nn.LayerNorm(n_text_state)
        self.router_proj = nn.Linear(n_text_state, 1)

        # 🔥 CRITICAL: safe router init
        with torch.no_grad():
            self.router_proj.weight.zero_()
            self.router_proj.bias.fill_(-10.0)

    def forward(
        self,
        x: torch.Tensor
    ) -> torch.Tensor:

        # Pre-LN
        x_norm = self.input_norm(x)

        # MHA
        y, _ = self.mha(x_norm, x_norm, x_norm, need_weights=False)

        # Residual + LN
        y = x + y
        y = self.output_norm(y)

        # Router (bounded input)
        y = self.router_norm(y)
        logits = self.router_proj(y)

        # Sigmoid gate
        gate = torch.sigmoid(logits)

        return gate


In [3]:
class LinearAttentionMem(nn.Module):
    def __init__(
        self,
        n_text_state: int,
        n_heads: int = 8,
        bias: bool = True,
    ):
        super().__init__()

        self.n_heads = n_heads
        self.n_text_state = n_text_state

        # Projections
        self.keys_proj   = nn.Linear(n_text_state, self.n_text_state * n_heads, bias=bias)
        self.values_proj = nn.Linear(n_text_state, self.n_text_state * n_heads, bias=bias)

        # foget Projections
        self.forget = nn.Linear(n_text_state, self.n_heads )

    def _split_heads(self, x: torch.Tensor) -> torch.Tensor:
        """ [B, S, n_text_state] → [B, S, H, D] """
        B, S, _ = x.shape
        return x.view(B, S, self.n_heads, self.n_text_state)


    def forward(self, x: torch.Tensor, M: torch.Tensor, router_weight: torch.Tensor, Mem_id):
        """
        Args:
            x     : [B, S, n_text_state]     current input chunk
            state : [B, H, D, D] or None  previous memory matrix (S_t = ∑ k v^T)

        Returns:
            output: [B, S, n_text_state]
            new_state: [B, H, D, D]
        """
        B, S, _ = x.shape

        # Project  K, V
        k = self.keys_proj(x)
        v = self.values_proj(x)

        #print("after projection",k.shape)

        # Split heads
        k = self._split_heads(k)                        # [B, S, H, D]
        v = self._split_heads(v)                        # [B, S, H, D]


        # Linear Attention Update
        for t in range(S):
            # Compute Forget
            forget = torch.sigmoid(self.forget(x))

            # Compute Memories Weight
            memory_weight = router_weight[:,t,Mem_id, None, None,None]

            # Update memory M
            M = forget[:,t,:, None, None]*M + memory_weight * torch.einsum('b h k, b h v -> b h k v', k[:, t], v[:, t])
            # Compute output at time t
            # o[:, t] = torch.einsum('b h k v, b h k -> b h v', M, q[:, t])

        return M

In [4]:
class MOM(nn.Module):
    def __init__(
        self,
        n_text_state,
        n_heads,
        num_memories,
        bias_term_adjust,
        LinearAttentionMem,
        topk=2

    ):
        super().__init__()

        self.n_text_state = n_text_state
        self.n_heads = n_heads
        self.num_memories = num_memories
        self.topk = topk
        self.bias_term_adjust = bias_term_adjust

        # Projections
        self.query_proj = nn.Linear(n_text_state, n_text_state * n_heads)

        # Output projection
        self.W = nn.Linear(n_text_state * n_heads, n_text_state)

        # Router
        self.router = nn.Linear(n_text_state, num_memories)

        # Create a LinearAttention module per memory slot
        self.mult_mem = nn.ModuleList(
            [LinearAttentionMem(n_text_state=self.n_text_state,
                                n_heads= self.n_heads) for _ in range(num_memories)]
        )

        self.softmax = nn.Softmax(dim=-1)

    def split_head(self, t):
        """
        [B, S, H*D] -> [B, S, H, D]
        """
        B, S, _ = t.shape
        return t.view(B, S, self.n_heads, self.n_text_state)

    def mixed_head(self, t):
        """
        [B, S, H, D]->[B, S, H*D]
        """
        B, S, _,_ = t.shape
        return t.view(B, S, self.n_heads*self.n_text_state)

    def forward(self, x, M,bias):
        """
        x: [B, S, n_text_state]
        M: [B, H*NumMem, K, V]
        """
        B, S,_ = x.shape
        total_token_routed = self.topk * S
        avg_load_per_expert = total_token_routed /self.num_memories
        # Compute projections
        q = self.query_proj(x)
        q = self.split_head(q)                # [B, S, H, D]
        _,_,H,D = q.shape

        # Router
        router_weight = self.router(x)                          # [B, S, num_memories]
        routing_weight, bias = self.bias_term_adjust(router_weight,
                                                     self.softmax,
                                                     avg_load_per_expert, self.num_memories, bias)

        # M emory Temp
        memory_outputs = []

        # Run selected memory modules only
        for mem_id in range(self.num_memories):
            # Extract prev mem for this memory: [B,H,K,V]
            M_t = M[:,mem_id*self.n_heads:(mem_id+1)*self.n_heads, :,:]

            # Call attention module
            M_t = self.mult_mem[mem_id](x, M_t, routing_weight, mem_id)

            # Update in Mix Memory
            M[:,mem_id*self.n_heads:(mem_id+1)*self.n_heads, :,:] = M_t

            memory_outputs.append(M_t)



        # Sum all memory contributions
        final_memory = torch.stack(memory_outputs, dim=0).sum(0)   # [B,H,K,V]

        o = torch.zeros((B, S, H, D),  dtype=torch.float32).to(device)

        for t in range(S):
            o[:, t] = torch.einsum('b h k v, b h d -> b h v', final_memory, q[:, t])



        # Final output projection
        final_output = self.W(self.mixed_head(o))

        # # Merge heads and project
        # B, S, H, D = final_output.shape
        # final_output = final_output.view(B, S, H * D)
        # final_output = self.atten_out_proj(final_output)

        return bias, M, final_output


In [2]:
B = 8
S = 186
n_text_state = 1280

In [5]:
# Dummy hidden states
x = torch.arange(B*S*n_text_state).float().reshape(B,S,n_text_state)
print("x =", x.shape)

x = torch.Size([8, 186, 1280])


In [6]:
n_heads = 8
num_memories = 8

In [7]:
import torch

In [9]:
mom = MOM(n_text_state=n_text_state,
          n_heads=n_heads,
          num_memories=num_memories,
          bias_term_adjust=bias_term_adjust,
          LinearAttentionMem=LinearAttentionMem)

In [10]:
mom(x)

torch.Size([8, 186, 8, 1280, 1280])
torch.Size([8, 186])


RuntimeError: The size of tensor a (1280) must match the size of tensor b (186) at non-singleton dimension 4

In [10]:
bias = torch.zeros((B, num_memories),  dtype=torch.float32)
router_weight = torch.zeros((B,S, num_memories),  dtype=torch.float32)
direction = torch.ones((num_memories),  dtype=torch.float32)

In [11]:
bias

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [16]:
(bias*direction).unsqueeze(1).shape

torch.Size([8, 1, 8])

In [44]:
(direction * bias).shape

torch.Size([2, 8])

In [20]:
router_weight

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])

In [40]:
direction* bias

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
router_weight + (bias*direction).unsqueeze(1)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [47]:
routing, bias = mom(x)

router weight torch.Size([2, 3, 8])
biasi torch.Size([2, 8])
direction torch.Size([8])
torch.Size([2, 3, 8])


In [49]:
routing.shape

torch.Size([3, 1280])

In [ ]:
bias.shape

torch.Size([3, 1280])

: 

router weight torch.Size([2, 3, 8])
biasi torch.Size([2, 8])
direction torch.Size([8])


RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 2

In [13]:
M = torch.zeros((B, n_heads*num_memories,n_text_state, n_text_state),  dtype=torch.float32)

In [14]:
M.shape

torch.Size([2, 64, 1280, 1280])

In [14]:
bias = torch.zeros(1, num_memories)

In [22]:
attn_route = MHARouting(n_text_state=n_text_state, n_heads=n_heads).to(device)

In [17]:
attn_route(x)

tensor([[[4.5398e-05],
         [4.5398e-05],
         [4.5398e-05]],

        [[4.5398e-05],
         [4.5398e-05],
         [4.5398e-05]]], device='cuda:0', grad_fn=<SigmoidBackward0>)

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

In [19]:
mem_out.shape

torch.Size([2, 3, 1280])

In [ ]:
(attn_route(x) * mem_out).shape

torch.Size([1, 3, 1280])

# Whisper

In [ ]:
import whisper
from whisper.tokenizer import get_tokenizer

In [ ]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("Voice2002.mp3")
audio = whisper.pad_or_trim(audio)


In [ ]:
# decode the audio
options = whisper.DecodingOptions()

In [ ]:
task = options.task

In [ ]:
model = whisper.load_model("turbo")

In [ ]:
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels)

In [ ]:
audio_encoded = model.encoder(mel.unsqueeze(0))

In [ ]:
n_audio = 1

In [ ]:
tokenizer = get_tokenizer(
        model.is_multilingual,
        num_languages=model.num_languages,
        language='ne',
        task=task,
    )

In [ ]:
sot_sequence = tokenizer.sot_sequence
initial_tokens = sot_sequence

In [ ]:
tokens = torch.tensor([initial_tokens]).repeat(n_audio, 1)

In [ ]:
dims = model.dims

In [ ]:
decoder = model.decoder
encoder = model.encoder

In [ ]:
mom = MOM(n_text_state=dims.n_text_state,
          n_heads=dims.n_text_head,
          num_memories=num_memories,
          bias_term_adjust=bias_term_adjust,
          LinearAttentionMem=LinearAttentionMem)

In [ ]:
attn_routing = MHARouting(n_text_state=dims.n_text_state, n_heads=dims.n_text_head)

In [ ]:
M = torch.zeros((B, dims.n_text_head*num_memories,n_text_state, n_text_state),  dtype=torch.float32)

In [ ]:
bias = torch.zeros(1, num_memories)

In [ ]:
tokens = torch.tensor([initial_tokens]).repeat(n_audio, 1)

In [ ]:
offset = 0

In [ ]:
x = (
    decoder.token_embedding(tokens)
    + decoder.positional_embedding[offset : offset + tokens.shape[-1]]
)


In [ ]:
x

tensor([[[-0.0020,  0.0147,  0.0041,  ..., -0.0132, -0.0074,  0.0077],
         [ 0.0137, -0.0045, -0.0039,  ..., -0.0249, -0.0281, -0.0224],
         [-0.0059,  0.0077,  0.0043,  ...,  0.0014, -0.0066,  0.0100]]],
       grad_fn=<AddBackward0>)

In [ ]:
decoder.blocks[0](x,audio_encoded)

tensor([[[-0.0055,  0.0126,  0.0213,  ..., -0.0383,  0.0058, -0.0107],
         [-0.0284, -0.0272, -0.0292,  ..., -0.0492, -0.0149,  0.0065],
         [-0.0269,  0.0116, -0.0012,  ...,  0.0001, -0.0066,  0.0236]]],
       grad_fn=<AddBackward0>)

In [ ]:
bias, M, x_mem_out = mom(x, M, bias)

In [16]:
mem_routing = attn_routing(x)

NameError: name 'attn_routing' is not defined

In [ ]:
mem_routing

tensor([[[1.],
         [1.],
         [1.]]], grad_fn=<SoftmaxBackward0>)

In [ ]:
final_mem_out = mem_routing*x_mem_out

In [ ]:
final_mem_out

tensor([[[ 0.0068,  0.0105, -0.0066,  ...,  0.0044,  0.0010,  0.0032],
         [ 0.0049,  0.0006, -0.0004,  ...,  0.0004,  0.0025,  0.0094],
         [ 0.0039,  0.0064, -0.0062,  ...,  0.0045,  0.0008,  0.0026]]],
       grad_fn=<MulBackward0>)

In [ ]:
M.shape

torch.Size([1, 64, 1280, 1280])

In [ ]:
x_mem_out

tensor([[[ 0.0012,  0.0109, -0.0040,  ..., -0.0051,  0.0021, -0.0034],
         [ 0.0084,  0.0188, -0.0085,  ..., -0.0085,  0.0020, -0.0049],
         [ 0.0038,  0.0099, -0.0060,  ..., -0.0045,  0.0016, -0.0032]]],
       grad_fn=<ViewBackward0>)

In [ ]:
x

tensor([[[-0.0020,  0.0147,  0.0041,  ..., -0.0132, -0.0074,  0.0077],
         [ 0.0137, -0.0045, -0.0039,  ..., -0.0249, -0.0281, -0.0224],
         [-0.0059,  0.0077,  0.0043,  ...,  0.0014, -0.0066,  0.0100]]],
       grad_fn=<AddBackward0>)

In [ ]:
x_attn = x + decoder.blocks[0].attn_ln(decoder.blocks[0].attn(x)[0])

In [ ]:
x = x_mem_out + x_attn

In [ ]:
x

tensor([[[-0.0350, -0.0535, -0.0444,  ...,  0.0806,  0.0280,  0.0547],
         [-0.0121, -0.0672, -0.0597,  ...,  0.0686,  0.0058,  0.0245],
         [-0.0362, -0.0610, -0.0482,  ...,  0.0995,  0.0284,  0.0593]]],
       grad_fn=<AddBackward0>)

In [ ]:
x_attn

tensor([[[-0.0095,  0.0055, -0.0073,  ..., -0.0049, -0.0004,  0.0134],
         [ 0.0063, -0.0141, -0.0159,  ..., -0.0162, -0.0216, -0.0164],
         [-0.0132, -0.0013, -0.0075,  ...,  0.0102,  0.0004,  0.0160]]],
       grad_fn=<AddBackward0>)

In [ ]:
model.decoder(tokens, audio_encoded)

tensor([[[ -4.0956,  -6.6814,  -5.4366,  ...,  -2.7340,  -1.9674,  -1.7424],
         [ -4.2348,  -5.4625,  -7.7658,  ...,  -6.2431,  -3.3840,  -5.8946],
         [ -7.9785,  -6.7670, -10.4535,  ...,  -9.1242,  -6.8272,  -9.5730]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
num_dec_blocks = model.decoder.blocks

In [ ]:
x

tensor([[[-0.0020,  0.0147,  0.0041,  ..., -0.0132, -0.0074,  0.0077],
         [ 0.0137, -0.0045, -0.0039,  ..., -0.0249, -0.0281, -0.0224],
         [-0.0059,  0.0077,  0.0043,  ...,  0.0014, -0.0066,  0.0100]]],
       grad_fn=<AddBackward0>)

In [ ]:
audio_encoded

tensor([[[-0.2836, -0.0312, -0.3517,  ..., -0.7805, -0.2554, -0.4209],
         [-0.2005, -0.1548,  0.1227,  ..., -0.1956,  0.0665,  0.2159],
         [-0.1250,  0.5094, -0.1653,  ..., -0.1024,  0.9331, -0.1645],
         ...,
         [-0.1345, -0.3504,  0.0552,  ..., -0.3738, -0.1667, -0.0632],
         [-0.2617,  0.1005, -0.1483,  ..., -0.3230,  0.0567,  0.1538],
         [-0.2639,  0.1818, -0.1569,  ..., -0.2866,  0.1973,  0.4889]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
decoder.blocks[0]

ResidualAttentionBlock(
  (attn): MultiHeadAttention(
    (query): Linear(in_features=1280, out_features=1280, bias=True)
    (key): Linear(in_features=1280, out_features=1280, bias=False)
    (value): Linear(in_features=1280, out_features=1280, bias=True)
    (out): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  (cross_attn): MultiHeadAttention(
    (query): Linear(in_features=1280, out_features=1280, bias=True)
    (key): Linear(in_features=1280, out_features=1280, bias=False)
    (value): Linear(in_features=1280, out_features=1280, bias=True)
    (out): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (cross_attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  (mlp): Sequential(
    (0): Linear(in_features=1280, out_features=5120, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=5120, out_features=1280, bias=True)
  )
  (mlp_ln): LayerNorm((1280,), eps=

In [ ]:
len(num_dec_blocks)

4

In [ ]:

M = nn.ParameterList([
    nn.Parameter(torch.zeros(len(num_dec_blocks),B, dims.n_text_head * num_memories,
                             dims.n_text_state, dims.n_text_state))
    for _ in range(len(num_dec_blocks))
])


: 

In [ ]:
M[0].shape

torch.Size([1, 160, 1280, 1280])

In [ ]:

mult_mom = nn.ModuleList(
    [MOM(
        n_text_state=dims.n_text_state,
        n_heads=dims.n_text_head,
        num_memories=num_memories,
        bias_term_adjust=bias_term_adjust,
        LinearAttentionMem=LinearAttentionMem
    ) for _ in range(len(num_dec_blocks))]
)


In [ ]:
for i,blocks in enumerate(decoder.blocks):
    print(i)
    x_attn = blocks.attn(x)
    x_norm = blocks.attn_ln(x)
    x_mem = mult_mom[i](x, M[i], bias[i])


    break

0
MultiHeadAttention(
  (query): Linear(in_features=1280, out_features=1280, bias=True)
  (key): Linear(in_features=1280, out_features=1280, bias=False)
  (value): Linear(in_features=1280, out_features=1280, bias=True)
  (out): Linear(in_features=1280, out_features=1280, bias=True)
)


# Transformer

In [1]:
from datasets import load_dataset, Audio
from transformers import WhisperFeatureExtractor, WhisperTokenizer,pipeline
from dataclasses import dataclass
from typing import Any, Dict, List, Union
#from transformers.models.whisper.modeling_whisper import WhisperDecoderLayer,WhisperConfig,WhisperDecoder

/home/bishwa/Unversity/Audio_LLM_Memory/audio_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import os
import inspect

In [3]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [4]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Nepali", task="transcribe")

In [11]:
%pwd

'/home/bishwa/Unversity/Audio_LLM_Memory/research'

In [12]:
os.listdir()

['trials.ipynb', 'experiments.ipynb']

In [5]:
os.chdir("..")

In [14]:
%pwd

'/home/bishwa/Unversity/Audio_LLM_Memory'

In [6]:
dataset = load_dataset(
    "csv",
    data_dir="Data/male-female-data/male-female-data",
    data_files="FemaleVoice.tsv",
    delimiter="\t"
)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio_id', 'sentence'],
        num_rows: 566
    })
})


In [7]:
dataset["train"][0]

{'audio_id': 'Voice8',
 'sentence': 'त्यसै गरी विश्वको कुनाकुनामा शान्तिको संदेश छरी अमर बन्न पुगेका शान्तिका अग्रदुत गौतम बुद्ध पनि मेरै देशका बासिन्दा हुन्।'}

In [7]:
dataset = dataset.map(lambda x: {"audio_path": f"Data/male-female-data/male-female-data/{x['audio_id']}.wav"})

In [14]:
dataset = dataset.cast_column("audio_path", Audio(sampling_rate=16000))


In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio_id', 'sentence', 'audio_path'],
        num_rows: 509
    })
    test: Dataset({
        features: ['audio_id', 'sentence', 'audio_path'],
        num_rows: 57
    })
})

In [19]:
dataset['train'][0]['audio_path']['array']

array([ 0.        ,  0.        ,  0.        , ..., -0.00133919,
       -0.00120211, -0.00121323], shape=(60558,), dtype=float32)

In [11]:
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['audio_id', 'sentence', 'audio_path'],
        num_rows: 509
    })
    test: Dataset({
        features: ['audio_id', 'sentence', 'audio_path'],
        num_rows: 57
    })
})

In [20]:
def preprocess(batch):
    waveform= batch['audio_path']['array']

    batch['input_feature'] = feature_extractor(waveform, sampling_rate=16000).input_features[0]

    batch['labels'] = tokenizer(batch['sentence']).input_ids

    return batch

In [21]:
dataset = dataset.map(
    preprocess,
    remove_columns=dataset["train"].column_names
)


Map: 100%|██████████| 57/57 [00:00<00:00, 148.38 examples/s]


In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_feature', 'labels'],
        num_rows: 509
    })
    test: Dataset({
        features: ['input_feature', 'labels'],
        num_rows: 57
    })
})

In [23]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [24]:
model = model.to(device)

In [21]:
encoder = model.model.encoder

In [18]:
encoder

WhisperEncoder(
  (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
  (embed_positions): Embedding(1500, 768)
  (layers): ModuleList(
    (0-11): 12 x WhisperEncoderLayer(
      (self_attn): WhisperAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (activation_fn): GELUActivation()
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
  )
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwis

In [22]:
decoder = model.model.decoder

In [17]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    feature_extractor: Any
    tokenizer: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [18]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [23]:
device = 'cpu'

In [24]:
audio = torch.tensor(dataset['train']['input_feature'][0]).to(device)

In [25]:
audio = audio.unsqueeze(0)

In [88]:
encoder = encoder.to(device)

In [26]:
audio_encoded = encoder(audio)

In [24]:
audio_encoded

BaseModelOutput(last_hidden_state=tensor([[[ 0.2865,  0.0503,  1.1654,  ..., -0.6479, -0.0893, -0.2226],
         [ 0.8401,  0.3607,  1.3514,  ..., -0.4716, -0.5387,  0.1990],
         [ 0.2941, -0.2209,  1.4717,  ..., -0.1742,  0.0741,  0.0448],
         ...,
         [ 0.3969, -0.2385, -0.2005,  ..., -0.7538,  0.2420, -0.7191],
         [-0.0612, -0.7443,  0.0374,  ..., -1.0508,  0.1516, -0.5307],
         [ 0.3845,  0.3259,  1.7717,  ..., -0.1833, -1.1009, -0.1235]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [27]:
target_label = torch.tensor(dataset['train']['labels'][0]).unsqueeze(0).to(device)

In [75]:
target_label.shape

torch.Size([1, 65])

In [28]:
target_embed = decoder.embed_tokens(target_label)

In [29]:
target_embed_pos = decoder.embed_positions(target_embed).unsqueeze(0)

In [34]:
target_embed_pos

tensor([[[ 0.0043,  0.0010,  0.0108,  ..., -0.0046, -0.0061,  0.0013],
         [ 0.0039,  0.0044,  0.0034,  ..., -0.0047, -0.0080,  0.0058],
         [ 0.0086,  0.0028,  0.0057,  ..., -0.0031, -0.0086,  0.0086],
         ...,
         [ 0.0048,  0.0081, -0.0010,  ...,  0.0039, -0.0059,  0.0053],
         [ 0.0023,  0.0096, -0.0002,  ...,  0.0078, -0.0047,  0.0017],
         [ 0.0038,  0.0057, -0.0011,  ...,  0.0087, -0.0082,  0.0048]]],
       device='cuda:0', grad_fn=<UnsqueezeBackward0>)

In [35]:
decoder.layers[0].self_attn(target_embed_pos)

(tensor([[[-0.0869,  0.0941,  0.0184,  ..., -0.0174,  0.2033,  0.0431],
          [-0.0834,  0.0950,  0.0180,  ..., -0.0186,  0.2038,  0.0409],
          [-0.0823,  0.0945,  0.0180,  ..., -0.0195,  0.2038,  0.0397],
          ...,
          [-0.0802,  0.0951,  0.0175,  ..., -0.0205,  0.2048,  0.0385],
          [-0.0802,  0.0951,  0.0175,  ..., -0.0205,  0.2048,  0.0385],
          [-0.0802,  0.0951,  0.0175,  ..., -0.0205,  0.2048,  0.0385]]],
        device='cuda:0', grad_fn=<ViewBackward0>),
 None)

In [36]:
target_embed.dim()

3

In [28]:
target_embed = decoder.embed_tokens(target_label)
target_embed_pos = decoder.embed_positions(target_embed)
if target_embed_pos.dim() != 3:
    target_embed_pos = target_embed_pos.unsqueeze(0)


In [48]:
target_embed_pos = target_embed_pos.to('cpu')

In [ ]:
attn_output,_ = blocks.self_attn(target_embed_pos)

In [63]:
attn_output = attn_output.to(device)

In [30]:
B = target_embed.shape[0]
num_memories = 8

In [31]:
config = model.config

In [42]:
config.decoder_attention_heads

12

In [109]:
len(decoder.layers)

12

In [32]:
M = torch.zeros(
                (len(decoder.layers),B,
                num_memories * config.decoder_attention_heads,
                config.d_model, config.d_model),
                device=target_embed_pos.device,
                dtype=target_embed_pos.dtype,
            )
bias = torch.zeros((len(decoder.layers),B,
        num_memories),
        device=target_embed_pos.device,
        dtype = target_embed_pos.dtype,
        )

In [112]:
M[0].shape

torch.Size([1, 96, 768, 768])

In [114]:
bias[0].shape

torch.Size([1, 8])

In [33]:
from MemoryModule.conponents.Mom import MOM
from MemoryModule.conponents.LinearAttention import LinearAttentionMem

In [52]:
device = 'cpu'

In [34]:
context_mom = MOM(
            n_text_state=config.d_model,
            n_heads=config.decoder_attention_heads,
            num_memories=num_memories,
            bias_term_adjust=bias_term_adjust,
            LinearAttentionMem=LinearAttentionMem,
        ).to(device)

In [35]:
attn_route = MHARouting(config.d_model).to(device)

memory_norm = nn.LayerNorm(config.d_model).to(device)

In [36]:
attn_routing = attn_route(target_embed_pos)

In [73]:
attn_routing.shape

torch.Size([1, 66, 1])

In [43]:
target_embed_pos.shape

torch.Size([1, 170, 768])

In [66]:
residual = target_embed_pos 

In [47]:
M.shape

torch.Size([1, 96, 768])

In [55]:
bias, M, mem_out = context_mom(target_embed_pos, M,bias)

In [57]:
M = memory_norm(M)

In [67]:
M.shape

torch.Size([1, 96, 768, 768])

In [39]:
for i, blocks in enumerate(decoder.layers[0:3]):
    residual = x
    attn_routing = attn_route(x)
    bias[i], M[i], mem_out = context_mom(x, M[i],bias[i])
    x,_ = blocks.self_attn(x)
    x = blocks.self_attn_layer_norm(x + residual + attn_routing*mem_out)
    x, _ = blocks.encoder_attn(x, audio_encoded.last_hidden_state)
    x = blocks.encoder_attn_layer_norm(x + residual)
    residual = x
    x = blocks.fc1(x)
    x = blocks.activation_fn(x)
    x = blocks.fc2(x)
    x = blocks.final_layer_norm(x + residual)
    

In [47]:
decoder

WhisperDecoder(
  (embed_tokens): Embedding(51865, 768, padding_idx=50257)
  (embed_positions): WhisperPositionalEmbedding(448, 768)
  (layers): ModuleList(
    (0-11): 12 x WhisperDecoderLayer(
      (self_attn): WhisperAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): WhisperAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (encoder

In [87]:
output_model = model(audio,  decoder_input_ids=target_label)

In [90]:
output_model.logits.shape

torch.Size([1, 148, 51865])

In [83]:
print(inspect.signature(model.forward))

(input_features: Optional[torch.FloatTensor] = None, attention_mask: Optional[torch.LongTensor] = None, decoder_input_ids: Optional[torch.LongTensor] = None, decoder_attention_mask: Optional[torch.LongTensor] = None, head_mask: Optional[torch.Tensor] = None, decoder_head_mask: Optional[torch.Tensor] = None, cross_attn_head_mask: Optional[torch.Tensor] = None, encoder_outputs: Optional[tuple[tuple[torch.FloatTensor]]] = None, past_key_values: Optional[transformers.cache_utils.Cache] = None, decoder_inputs_embeds: Optional[tuple[torch.FloatTensor]] = None, decoder_position_ids: Optional[tuple[torch.LongTensor]] = None, labels: Optional[torch.LongTensor] = None, use_cache: Optional[bool] = None, output_attentions: Optional[bool] = None, output_hidden_states: Optional[bool] = None, return_dict: Optional[bool] = None, cache_position: Optional[torch.LongTensor] = None) -> Union[tuple[torch.Tensor], transformers.modeling_outputs.Seq2SeqLMOutput]


In [ ]:
# WhisperLayer

In [19]:
model

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [ ]:
model.model.decoder = WhisperDecoderWithSharedMemory(
        model.config,
        num_memories=8,
        bias_term_adjust=bias_term_adjust,
        LinearAttentionMem=LinearAttentionMem
    )

In [98]:
model.model.decoder

WhisperDecoderWithSharedMemory(
  (embed_tokens): Embedding(51865, 768, padding_idx=50257)
  (embed_positions): WhisperPositionalEmbedding(448, 768)
  (layers): ModuleList(
    (0-11): 12 x WhisperDecoderMemoryLayer(
      (self_attn): WhisperAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): WhisperAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=False)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)


In [99]:
config = model.config

In [100]:
config

WhisperConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "dtype": "float32",
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_feature_prob": 0.0,
  "mask_time_length": 10,
  "mask_time_min_masks": 2,
  "mask_time_prob": 0.05

In [27]:
audio = torch.tensor(dataset['train']['input_feature'][1])

In [28]:
audio

tensor([[-0.4938, -0.4938, -0.4938,  ..., -0.4938, -0.4938, -0.4938],
        [-0.4938, -0.4938, -0.4938,  ..., -0.4938, -0.4938, -0.4938],
        [-0.4938, -0.4938, -0.4938,  ..., -0.4938, -0.4938, -0.4938],
        ...,
        [-0.4938, -0.4938, -0.4938,  ..., -0.4938, -0.4938, -0.4938],
        [-0.4938, -0.4938, -0.4938,  ..., -0.4938, -0.4938, -0.4938],
        [-0.4938, -0.4938, -0.4938,  ..., -0.4938, -0.4938, -0.4938]])

In [29]:
audio = audio.unsqueeze(0)

In [30]:
target_label = torch.tensor(dataset['train']['labels'][1]).unsqueeze(0)

In [32]:
target_label.shape

torch.Size([1, 85])

In [34]:
result_model = model(audio, decoder_input_ids=target_label)

In [35]:
final_output = torch.argmax(result_model.logits, dim=-1)[0]

In [36]:
final_output

tensor([50313, 50358, 50363,  8485,  8703,  8485,   105, 17937,  3941,   223,
        35082, 17937,  8485,   103, 25411, 25411, 45938,  8485,  3941,   103,
        25411, 33279, 41858,  8485,   237, 27099,   105, 27099,  3941,   113,
         3941,    94,  3941,    99, 33926, 33926,  8485,   250, 35082, 27099,
        36158, 31881,  8485,   108,  3941, 25411, 25411, 27099,  3941,    99,
        17937, 31970, 33926, 48449, 46758, 17937, 48521, 31881,  8485,   246,
         8485,   231,  3941,   244, 17937, 35082, 49316, 25411, 25411, 27099,
         3941,    98,  3941,  8485,   105, 44500,   230, 31970, 33926,  8485,
          248, 17937,   224, 50257, 50257])

In [37]:
tokenizer.decode(final_output, skip_special_tokens=True)

' �� बाँना पररस �परिक ए्�्वडदोो जन्ती र�रर्दा को मलामी घ उखान सरर्थ� बह� को चा�'

In [38]:
tokenizer.decode(target_label[0], skip_special_tokens=True)

'यो भावना पारस्परिक ‘जिउँदको जन्ती र मार्दाको मलामी’ उखान सार्थक भएको छ।'

In [40]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=tokenizer,
    feature_extractor=feature_extractor,
    torch_dtype=torch.float32,
    device='cpu'
)

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu


In [60]:
audio.device

device(type='cpu')

In [63]:
audio.shape

torch.Size([1, 80, 3000])

In [41]:
result = pipe("Data/male-female-data/male-female-data/Voice706.wav")

`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


In [42]:
result

{'text': ' यो ब्हाँना पारस पर एक जिव्दो को जन्ती रँ मर्दा को मलामी उखान सर्थक बहे को चा.'}

In [ ]:


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-modify",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


ValueError: --load_best_model_at_end requires the save and eval strategy to match, but found
- Evaluation strategy: IntervalStrategy.NO
- Save strategy: SaveStrategy.STEPS

In [ ]:
decoder

WhisperDecoder(
  (embed_tokens): Embedding(51865, 1280, padding_idx=50257)
  (embed_positions): WhisperPositionalEmbedding(448, 1280)
  (layers): ModuleList(
    (0-31): 32 x WhisperDecoderLayer(
      (self_attn): WhisperAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): WhisperAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
   

In [ ]:
decoder.layers

ModuleList(
  (0-31): 32 x WhisperDecoderLayer(
    (self_attn): WhisperAttention(
      (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
      (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
      (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
      (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
    )
    (activation_fn): GELUActivation()
    (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (encoder_attn): WhisperAttention(
      (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
      (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
      (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
      (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
    )
    (encoder_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (fc1): Linear(in_features=1280, out_features=5120, bias=True)
    (fc2): Linear(in

In [4]:
from MemoryModule.conponents.modeling_whisper import ConditionalGeneration

In [7]:
from transformers import WhisperForConditionalGeneration


In [5]:
model = ConditionalGeneration.from_pretrained("openai/whisper-small")

In [6]:
model

ConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_l